# 1st Prototype

In [1]:
#Library to make GUI
from tkinter import messagebox
import tkinter as tk

import requests 
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

# Create URL
url="https://www.billboard.com/charts/hot-100/"

# Create a request
results=requests.get(url, headers = {"Accept-Language": "en-US"})

# Result in string
#results.text

# We use BeautifulSoup to extract specific results
soup=BeautifulSoup(results.content,'html.parser')

# Show results in html mode
#print(soup.prettify())

# Strip=True => You can tell Beautiful Soup to strip whitespace from the beginning and end of each bit of text
# Try get the first name of song
soup.select('h3.c-title.a-no-trucate')[0].get_text(strip=True)

# Try get the first artist of song
soup.select('span.c-label.a-no-trucate')[0].get_text(strip=True)

# Calculate the len of our select
len_songs=len(soup.select('h3.c-title.a-no-trucate'))

# Create variables to store results
song = []
artist = []

# Create a for to pick the 100 best songs
for x in range(len_songs):
    song.append(soup.select('h3.c-title.a-no-trucate')[x].get_text(strip=True))
    artist.append(soup.select('span.c-label.a-no-trucate')[x].get_text(strip=True))
    
# Import pandas to create a data frame
top=pd.DataFrame({'Song':song, 'Artist':artist})
top

# Import numpy to add a Rank
top['Rank']=np.arange(len(top))+1
top


,Song,Artist,Rank
0,We Don't Talk About Bruno,"Carolina Gaitan, Mauro Castillo, Adassa, Rhenz...",1
1,Heat Waves,Glass Animals,2
2,abcdefu,GAYLE,3
3,Easy On Me,Adele,4
4,Super Gremlin,Kodak Black,5
...,...,...,...
95,Closer,Saweetie Featuring H.E.R.,96
96,Cold December,Rod Wave,97
97,Get Into It (Yuh),Doja Cat,98
98,All Too Well (Taylor's Version),Taylor Swift,99


In [12]:
# Create a window
window = tk.Tk()

#Change size os window
window.geometry(f'412x150')

# Title
title = tk.Label(window, text = 'Enter a song and/or an artist to see if its in the top 100', bg = 'gray51')
title.pack(fill = tk.X)

#Take song
title_song = tk.Label(window, text = 'Enter a song')
title_song.pack(fill = tk.X)
input_song = tk.Entry(window)
input_song.pack()

#Take artist
title_artist = tk.Label(window, text = 'Enter a artist')
title_artist.pack(fill = tk.X)
input_artist = tk.Entry(window)
input_artist.pack()

# Functions
def get_data(input_song,input_artist,top):
    # Transform to string
    songs = input_song.get()
    artists = input_artist.get()
    if len(songs) == 0 | len(artists) == 0:
        messagebox.showwarning('Error','You need to enter at least one song or artist.')
    else:
        # Put all in lower
        result = top.applymap(lambda s:s.lower() if type(s) == str else s)
        songs = songs.lower()
        artists = artists.lower()
        
        # Check if there are some result 
        check = result[(result['Song']==songs)|(result['Artist']==artists)]
        check
        
        # Create a new dataframe without result to avoid repeat
        result = top.drop(labels=check.index, axis=0)
        
        #Create a new dataframe with a random sample to share
        result = result.sample()
        song_res = result['Song'].item()
        artist_res = result['Artist'].item()
        
        # Show the results
        window.geometry("")
        if len(check) == 0:
            res1 = tk.Label(window, text = 'Bad luck, your song or artist is not in the top 100, maybe you like this one too')
            res1.pack(fill = tk.X)
            res2 = tk.Label(window, text = f'Song:{song_res}- Artist:{artist_res}')
            res2.pack(fill = tk.X)
        else:
            res1 = tk.Label(window, text = 'Wow, your song or artist is in the top 100, maybe you like this one too')
            res1.pack(fill = tk.X)
            res2 = tk.Label(window, text = f'Song:{song_res}- Artist:{artist_res}')
            res2.pack(fill = tk.X)



# Bu
button_search = tk.Button(window, text = 'Search', command = lambda: get_data(input_song,input_artist,top))
button_search.pack()  

#That comand open the window
window.mainloop()